In [ ]:

names = (names if isinstance(names, (list, np.ndarray))
                      else RawCommittees.objects.values_list('committee_name', flat=True))

        self.pac_type_tuples = RawCommittees.objects.values_list('committee_type', 'committee_subtype')
        self.labels = np.array(list(labels or self.pac_type_tuples))
        # self.labels = [', '.join(str(s) for s in pt) for pt in self.pac_type_tuples]
        self.labels = np.array([str(lbl) for lbl in self.labels])
        self.label_set = sorted(np.unique(self.labels))
        self.label_dict = dict(list(zip(self.label_set, range(len(self.label_set)))))
        self.label_ints = np.array([self.label_dict[label] for label in self.labels])
        if self.verbosity > 1:
            print(pd.Series(self.labels))
        if self.verbosity > 0:
            print(np.unique(self.labels))

        self.tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 1), stop_words='english')
        self.tfidf_matrix = self.tfidf.fit_transform(self.names)
        if verbosity > 1:
            print(self.tfidf.get_feature_names())
        self.train_tfidf, self.test_tfidf, self.train_labels, self.test_labels = train_test_split(
            self.tfidf_matrix, self.label_ints, test_size=.25)
        # alpha: learning rate (default 1e-4, but other TFIDF classifier examples use 1e-5 to 1e-6)
        # penalty: 'none', 'l2', 'l1', or 'elasticnet'  # regularization penalty on the feature weights
        self.svm_matrix = self.fit(self.train_tfidf, self.train_labels)
        if verbosity > 0:
            print(self.score(self.train_tfidf, self.train_labels))